# Distributed model inference using TensorFlow Keras
From: https://docs.databricks.com/_static/notebooks/deep-learning/keras-metadata.html

In [ ]:
import os
import shutil
import time
import pandas as pd
from PIL import Image
import numpy as np
import uuid
 
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
 
from pyspark.sql.functions import col, pandas_udf, PandasUDFType

In [ ]:
file_name = "image_data.parquet"
output_file_path = "predictions"

### Prepare trained model and data for inference

Load the ResNet-50 Model and broadcast the weights.

In [ ]:
model = ResNet50()
bc_model_weights = sc.broadcast(model.get_weights())

Load the data and save the datasets to one Parquet file.

In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin=dataset_url,
                                   fname='flower_photos',
                                   untar=True)
data_dir = pathlib.Path(data_dir)

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
import os
files = [os.path.join(dp, f) for dp, dn, filenames in os.walk(data_dir) for f in filenames if os.path.splitext(f)[1] == '.jpg']
files = files[:2048]
len(files)

In [ ]:
print(data_dir)

In [ ]:
image_data = []
for file in files:
    img = Image.open(file)
    img = img.resize([224, 224])
    data = np.asarray(img, dtype="float32").reshape([224*224*3])

    image_data.append({"data": data})

pandas_df = pd.DataFrame(image_data, columns=['data'])
pandas_df.to_parquet(file_name)
# os.makedirs(dbfs_file_path)
# shutil.copyfile(file_name, dbfs_file_path+file_name)

### Load the data into Spark DataFrames

In [ ]:
from pyspark.sql.types import *
df = spark.read.parquet(file_name)
print(df.count())

In [ ]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")
spark.conf.set("spark.sql.parquet.columnarReaderBatchSize", "1024")

In [ ]:
assert len(df.head()) > 0, "`df` should not be empty" # This line will fail if the vectorized reader runs out of memory

### Run model inference via pandas UDF

In [ ]:
def parse_image(image_data):
    image = tf.image.convert_image_dtype(
        image_data, dtype=tf.float32) * (2. / 255) - 1
    image = tf.reshape(image, [224, 224, 3])
    return image

In [ ]:
@pandas_udf(ArrayType(FloatType()), PandasUDFType.SCALAR_ITER)
def predict_batch_udf(image_batch_iter):
    batch_size = 64
    model = ResNet50(weights=None)
    model.set_weights(bc_model_weights.value)
    for image_batch in image_batch_iter:
        images = np.vstack(image_batch)
        dataset = tf.data.Dataset.from_tensor_slices(images)
        dataset = dataset.map(parse_image, num_parallel_calls=8).prefetch(
            5000).batch(batch_size)
        preds = model.predict(dataset)
        yield pd.Series(list(preds))

In [ ]:
%%time
predictions_df = df.select(predict_batch_udf(col("data")).alias("prediction"))
predictions_df.write.mode("overwrite").parquet(output_file_path)

In [ ]:
result_df = spark.read.parquet(output_file_path)
result_df.show(truncate=120)

### Model inference using sparkext

In [ ]:
from pyspark.sql.functions import struct
from sparkext.tensorflow import model_udf

In [ ]:
model = ResNet50()

In [ ]:
predict_batch_udf = model_udf(model, batch_size=64)

In [ ]:
%%time
predictions_df = df.select(predict_batch_udf(struct("data")).alias("prediction"))
predictions_df.write.mode("overwrite").parquet(output_file_path + "_sparkext")

In [ ]:
result_df = spark.read.parquet(output_file_path)
result_df.show(truncate=120)

### Model inference using MLFlow

In [ ]:
import mlflow
from mlflow.models.signature import infer_signature  #, ModelSignature
# from mlflow.types.schema import Schema, TensorSpec

In [ ]:
train_images = np.vstack(pandas_df['data'].head(10).to_numpy()).reshape(-1,224,224,3)
predictions = model.predict(train_images)

In [ ]:
signature = infer_signature(train_images, model.predict(train_images))
signature

In [ ]:
tf.keras.models.save_model(model, "resnet50_model")

In [ ]:
mlflow.tensorflow.save_model(tf_saved_model_dir="resnet50_model", 
                             tf_meta_graph_tags=["serve"], 
                             tf_signature_def_key="serving_default",
                             signature=signature,
                             path="resnet50_mlflow")

In [ ]:
predict_batch_udf = mlflow.pyfunc.spark_udf(spark, model_uri="resnet50_mlflow", result_type="array<float>")

In [ ]:
%%time
predictions_df = df.select(predict_batch_udf(struct("data")).alias("prediction"))
predictions_df.write.mode("overwrite").parquet(output_file_path + "_mlflow")

### Model inference using Triton UDF

In [ ]:
from sparkext.triton import model_udf

In [ ]:
model = ResNet50()

In [ ]:
predict_batch_udf = model_udf(model, batch_size=64)

In [ ]:
%%time
predictions_df = df.select(predict_batch_udf(col("data")).alias("prediction"))
predictions_df.write.mode("overwrite").parquet(output_file_path + "_sparkext")

In [ ]:
result_df = spark.read.parquet(output_file_path)
result_df.show(truncate=120)

### Model inference using Spark DL API

In [1]:
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import struct
from pyspark.sql.types import ArrayType, FloatType

In [2]:
def predict_batch_fn():
    import tensorflow as tf
    from tensorflow.keras.applications.resnet50 import ResNet50
    model = ResNet50()
    def predict(inputs):
        return model.predict(inputs)
    return predict

In [3]:
classify = predict_batch_udf(predict_batch_fn,
                             input_tensor_shapes=[[-1, 224, 224, 3]],
                             return_type=ArrayType(FloatType()),
                             batch_size=50)

In [4]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")
spark.conf.set("spark.sql.parquet.columnarReaderBatchSize", "1024")

In [5]:
df = spark.read.parquet("image_data.parquet")

In [6]:
predictions = df.select(classify(struct("data")).alias("prediction"))
predictions.show(truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                              prediction|
+------------------------------------------------------------------------------------------------------------------------+
|[6.480268E-9, 1.7902835E-8, 7.2895295E-10, 1.12287755E-10, 4.777694E-9, 2.2727333E-8, 3.2138614E-10, 1.1390412E-7, 1....|
|[4.272546E-10, 1.0472946E-9, 3.108161E-11, 1.3740103E-11, 9.3929885E-11, 1.0310459E-9, 6.280175E-12, 3.2855128E-9, 2....|
|[7.145851E-5, 8.087193E-5, 0.15172856, 8.129367E-4, 8.880329E-4, 1.0974148E-4, 9.9298115E-5, 1.1457807E-4, 3.2580836E...|
|[7.687008E-6, 1.7460868E-4, 1.17645766E-4, 1.8629865E-4, 4.112672E-4, 3.5050482E-4, 1.04423976E-4, 1.8672385E-5, 6.10...|
|[1.9156582E-6, 3.662862E-5, 4.240978E-5, 1.1411563E-5, 1.6928356E-4, 7.110861E-5, 2.9225656E-4, 6.846987E-6, 2.140841...|
|[9.34794E-6, 1.

### Using Triton Server

#### Start Triton Server on each executor

In [7]:
num_executors = 1

nodeRDD = sc.parallelize(list(range(num_executors)), num_executors)

def start_triton(it):
    import docker
    import time
    import tritonclient.grpc as grpcclient
    
    client=docker.from_env()
    containers=client.containers.list(filters={"name": "spark-triton"})
    if containers:
        print(">>>> containers: {}".format([c.short_id for c in containers]))
    else:
        container=client.containers.run(
            "nvcr.io/nvidia/tritonserver:22.07-py3", "tritonserver --model-repository=/models",
            detach=True,
            device_requests=[docker.types.DeviceRequest(device_ids=["0"], capabilities=[['gpu']])],
            environment=[
                "TRANSFORMERS_CACHE=/cache"
            ],
            name="spark-triton",
            network_mode="host",
            remove=True,
            shm_size="512M",
            volumes={
                "/home/leey/devpub/leewyang/sparkext/examples/models": {"bind": "/models", "mode": "ro"},
                "/home/leey/huggingface/cache": {"bind": "/cache", "mode": "rw"}
            }
        )
        print(">>>> starting triton: {}".format(container.short_id))

        # wait for triton to be running
        time.sleep(15)
        client = grpcclient.InferenceServerClient("localhost:8001")
        ready = False
        while not ready:
            try:
                ready = client.is_server_ready()
            except Exception as e:
                time.sleep(5)

    return [True]

nodeRDD.mapPartitions(start_triton).collect()

[True]

#### Run inference

In [8]:
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import struct
from pyspark.sql.types import ArrayType, FloatType

In [9]:
def triton_fn(triton_uri, model_name):
    import numpy as np
    import tritonclient.grpc as grpcclient
    
    np_types = {
      "BOOL": np.dtype(np.bool8),
      "INT8": np.dtype(np.int8),
      "INT16": np.dtype(np.int16),
      "INT32": np.dtype(np.int32),
      "INT64": np.dtype(np.int64),
      "FP16": np.dtype(np.float16),
      "FP32": np.dtype(np.float32),
      "FP64": np.dtype(np.float64),
      "FP64": np.dtype(np.double),
      "BYTES": np.dtype(object)
    }

    client = grpcclient.InferenceServerClient(triton_uri)
    model_meta = client.get_model_metadata(model_name)
    
    def predict(inputs):
        if isinstance(inputs, np.ndarray):
            # single ndarray input
            request = [grpcclient.InferInput(model_meta.inputs[0].name, inputs.shape, model_meta.inputs[0].datatype)]
            request[0].set_data_from_numpy(inputs.astype(np_types[model_meta.inputs[0].datatype]))
        else:
            # dict of multiple ndarray inputs
            request = [grpcclient.InferInput(i.name, inputs[i.name].shape, i.datatype) for i in model_meta.inputs]
            for i in request:
                i.set_data_from_numpy(inputs[i.name()].astype(np_types[i.datatype()]))
        
        response = client.infer(model_name, inputs=request)
        
        if len(model_meta.outputs) > 1:
            # return dictionary of numpy arrays
            return {o.name: response.as_numpy(o.name) for o in model_meta.outputs}
        else:
            # return single numpy array
            return response.as_numpy(model_meta.outputs[0].name)
        
    return predict

In [10]:
classify = predict_batch_udf(triton_fn,
                             triton_uri="localhost:8001",
                             model_name="resnet_50",
                             input_tensor_shapes=[[-1, 224, 224, 3]],
                             return_type=ArrayType(FloatType()),
                             batch_size=50)

In [11]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")
spark.conf.set("spark.sql.parquet.columnarReaderBatchSize", "1024")

In [12]:
df = spark.read.parquet("image_data.parquet")

In [13]:
predictions = df.select(classify(struct("data")).alias("prediction"))
predictions.show(truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                              prediction|
+------------------------------------------------------------------------------------------------------------------------+
|[6.4928676E-9, 1.7909096E-8, 7.3115214E-10, 1.1236475E-10, 4.781937E-9, 2.2758408E-8, 3.216114E-10, 1.1415651E-7, 1.2...|
|[4.2570847E-10, 1.0446737E-9, 3.1041343E-11, 1.3694482E-11, 9.367803E-11, 1.0271425E-9, 6.2756176E-12, 3.2876208E-9, ...|
|[7.170488E-5, 8.105519E-5, 0.15266123, 8.1522577E-4, 8.923165E-4, 1.10131026E-4, 9.962701E-5, 1.1484663E-4, 3.260079E...|
|[7.697718E-6, 1.748849E-4, 1.17513E-4, 1.8603323E-4, 4.1148433E-4, 3.5065506E-4, 1.04553015E-4, 1.8722472E-5, 6.11295...|
|[1.9293225E-6, 3.68381E-5, 4.2936168E-5, 1.1517309E-5, 1.7104512E-4, 7.176816E-5, 2.9497486E-4, 6.8992535E-6, 2.15252...|
|[9.326582E-6, 1

#### Stop Triton Server on each executor

In [14]:
def stop_triton(it):
    import docker
    import time
    
    client=docker.from_env()
    containers=client.containers.list(filters={"name": "spark-triton"})
    print(">>>> stopping containers: {}".format([c.short_id for c in containers]))
    if containers:
        container=containers[0]
        container.stop(timeout=120)

    return [True]

nodeRDD.mapPartitions(stop_triton).collect()

[True]

In [15]:
spark.stop()